# uSearch 
This search engine will allow users to find other Twitter users to follow. The user may search based on the tweet content. Users will be able to enter a query that will be matched against our Twitter corpus and output ranked results that best fit the user's query. uSearch also features a query option of allowing the user to search for tweets from a specified user by starting their query with an '@' symbol.  

# Approach 
Our approach was to read in tweets from a pre-defined dataset, normalize the data, create an inverted index, apply a ranking algorithm, and output results. The user will be asked whether they are searching for a tweets matching their query content or for a specific user's tweets. Tweets are entered into the inverted index using the stemmed equivalence of the words in the tweet. For stemming the words, we used the nltk module's PorterStemmer library.

# Key Insights 
a) A larger corpus allows for more diverse results. <br>
b) Clients prefered results that are mid-sized. The constant values are 2.5 for the k1 constant and 0.25 for the b constant. <br>
c) We discovered our search engine was limited to returing results for exact string matches in the user's query. We decided to stem the uesr queries and our inverted index to provide a wider variety of results. For example, the queries for 'happy' and 'happiness' will return the same results because they both derive from the root 'happi' in our inverted index. 

# Imports
All needed libraries for this search engine to be functional.

In [1]:
import re
import csv
import math
import collections
import pandas as pd
import bokeh
from nltk import PorterStemmer

# Get Twitter Docs
Returns all corpus documents from a given filename 

In [2]:
def getTwitterDocs(filename):
	with open(filename, 'rb') as csvfile:
		r = csv.reader(csvfile, delimiter=',', quotechar='"')
		docs = [ (x[2], x[4], x[5])  for x in r]

	return docs

# Stemming 

In [3]:
def stem_tweet(tweet):
    stemmed_tweet = ""
    split_tweet = tweet.split(' ')
    for word in split_tweet:
        stemmed_tweet += PorterStemmer().stem_word(word) + " "
    stemmed_tweet = stemmed_tweet.strip(' ')
    return stemmed_tweet
#print stem_tweet("happiness indentifiable")
#print stem_tweet("happy indentity")
        

# Erase Link
This function is part of the normalization effort. A full length tweet will be inserted, and a tweet without a link will be output. 

In [4]:
def erase_link(tweet):
    inx = tweet.find('http')
    #Check to see if tweet contains a link
    if inx != -1:
        #Find white space
        sp_inx = tweet.find(" ", inx)
        #Check if there is a white space after the link
        if sp_inx != -1:
            new_tweet = tweet[0:inx] + tweet[sp_inx + 1:]
        else:
            new_tweet = tweet[0:inx]
        return new_tweet
    return tweet

# Normalize
This function will normalize a tweet passed in. The output will be the tweet without any special characters, except a '@,' in all lowercase.

In [5]:
def normalize(tweet):
    #erase link
    normalizedTweet = erase_link(tweet)
    #print normalizedTweet
    normalizedTweet = re.compile('[^a-zA-Z@]').sub(' ', normalizedTweet)
    normalizedTweet = normalizedTweet.lower()
    normalizedTweet = " ".join(normalizedTweet.split())
    return normalizedTweet


This portion of the program will open a csv file containing all tweets that will comprise our corpus. Three lists will be created: one for the dates, one for the usernames, and one for the tweets. These three lists are parallel to each other.

In [6]:
with open("twitter-data/smallData.csv", 'rb') as csvfile:
    r = csv.reader(csvfile, delimiter=',', quotechar='"')
    docs = [ (x[2], x[4], x[5])  for x in r]
    
tweet_date = [ d[0] for d in docs ] #Tweet dates
tweet_user = [ d[1] for d in docs ] #Tweet users
tweet_text = [ d[2] for d in docs ] #Tweet text

#Normalize each tweet 
tweet_text = map(normalize, tweet_text)


# Create Corpus 
This function will generate a corpus as a dictionary. The key value is a the document id, while the value is the date, username, and text

In [7]:
def create_corpus():
    corpus = {}
    for x in range(0,len(tweet_text)):
        corpus[x] = [tweet_date[x], tweet_user[x],tweet_text[x]]
        #print corpus[x]



# Inverted Indexs

For the twitter data, there will be two inverted indexes. The first inverted index will have the username as the key and the tweets that correspond to that username as the values. The second inverted index will be the word index for our tweets; the key will be a word and the values will be the id of the documents that contain that word.

# Word Dictionary Reversed Index

The reversed index for the words in the corpus will be created by going through each word in each document. The reveresed index will allow for a faster retrieval of relevant documents. 

In [8]:
def create_inverted_index(corpus):
    idx = {}
    
    for i, doc in enumerate(corpus):
        # Stem document (tweet)
        stemmed_document = stem_tweet(doc)
        # Iterate through each word in the document
        for word in stemmed_document.split():
        #for word in doc.split():
            if word in idx:
                # Update the document's term frequency
                if i in idx[word]:
                    idx[word][i] += 1
                # Add the document to the word index
                else:
                    idx[word][i] = 1;
            # Add the word to the reversed index
            else:
                idx[word] = {i:1}
    
    return idx


In [9]:
'''
test_users = ["vcu451", "chadfu", "SIX15"]
test_corpus = ["reading my kindle2  love it lee childs is good read", 
               "ok, first assesment of the kindle2 ...it fucking rocks", 
               "fuck this economy I hate aig and their non loan given asses"]
'''

idx = create_inverted_index(tweet_text)


# User Tweet Index

The user tweet index will allow for the all tweets(documents) that belong to a username to be retrieved quickly.

In [10]:
def create_user_index(users):
    
    user_tweets = {}
    
    # Go through each of the tweets in the corpus
    for i in range( len(users) ):
        # When user already exists, add the document id to the existing user tweet list
        if users[i] in user_tweets:
            user_tweets[users[i] ].append(i)
        # Otherwise, creat a new list with the document id
        else:
            user_tweets[users[i] ] = [i]
            
    return user_tweets

In [11]:
user_tweet_index = create_user_index(tweet_user)

#user_tweet_index

# Document Ranking
To rank the documents, we will be implementing two different ranking algorithms: TF-IDF and BM25. 

# TF-IDF
The TF-IDF (term frequency–inverse document frequency) ranking algorithm ranks documents based on the term frequency of the words in the query in relation to the words in the documents.

In [12]:
def print_results(results, n, head=True):
    ''' Helper function to print results
    '''
    if head:
        print('\nTop %d from recall set of %d items:' % (n, len(results) ) )
        for r in results[:n]:
            print('\t%0.2f - %s - %s' % (r[0], r[2],tweet_text[r[1]]))
    else:
        print('\nTop %d from recall set of %d items:' % (n, len(results) ) )
        for r in results[:n]:
            print('\t%0.2f - %s - %s' % (r[0],r[2], tweet_text[r[1]]))

In [13]:
def idf(term, idx, n):
    # term - the term that is being scored
    # idx - the reversed index on the terms in the corpus
    # n - the number of docments the term appears in
    return math.log(float(n) / (1 + len(idx[term])))
    
#print(idf('how', idx, len(tweet_user)))
#print(idf('hate', idx, len(tweet_user)))
#print(idf('sleep', idx, len(tweet_user)))
#print(idf('whoopi', idx, len(tweet_user)))
#print(idf('monkeys', idx, len(tweet_user)))

In [14]:
def get_results_tfidf(qry, idx, n):
    score = collections.Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    sorted_results= sorted(results, key=lambda t : t[0] * -1)
    return sorted_results
#results = get_results_tfidf('monkeys', idx, len(tweet_user))
#results = get_results_tfidf('hate', idx, len(tweet_user))
#results = get_results_tfidf('sleep', idx, len(tweet_user))
results = get_results_tfidf('hate', idx, len(tweet_user))

#print_results(results, 10)

# BM25
Implement the BM25 ranking algorithm to rank our results. This ranking algorithm is the one we will be using for the final version of our search engine.

In [22]:
def get_results_bm25(idx, qry, k1=2.5, b=0.25):

    score = collections.Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # f - the number of times the term appears in the document
                f = float(idx[term][doc])
                # s - the BM25 score for this (term, docuemnt) pair
                s = i * ( (f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc] ) / d_avg) ) ) ) )
                score[doc] += s
                
    results = []
    for x in [ [r[0], r[1], tweet_user[r[0]] ] for r in zip(score.keys(), score.values() )]:
        if x[1] > 0:
            results.append([ x[1], x[0], x[2]])
            
    sorted_results  = sorted(results, key=lambda t: t[0] * -1)
    return sorted_results

# Visualize the Effectiveness of BM25

In [16]:
from bokeh.plotting import output_notebook, show
from bokeh.charts import Scatter

In [23]:
idx = create_inverted_index(tweet_text)
results = get_results_bm25(idx, stem_tweet('i hate tomatos'), k1=2.5, b=0.75)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(tweet_text[x[1]].split()) for x in results]})
output_notebook()
p = Scatter(df, x='score', y='length')
show(p)

Loading BokehJS ...

# Get content based on username
This function will retrieve tweets in the corpus that belong to the specified user. These tweets will be ordered by date. 

In [18]:
def print_results_username(username_index, user_name):
    if(user_name in username_index):
        # All of the documents that correspond to the username 
        docs = username_index[user_name]
        # For every doc, print date, username, and tweet 
        for doc in docs:
            print tweet_date[doc] + " " + tweet_user[doc] + " " + tweet_text[doc]
    else:
        print "Username does not exist"
    
results = create_user_index(tweet_user)
print_results_username(results, 'SimpleManJess')

Tue Jun 02 04:29:16 UTC 2009 SimpleManJess ncaa baseball super regional rams club
Tue Jun 02 04:29:17 UTC 2009 SimpleManJess baseballamerica com blog baseball america prospects blog blog


# Determine User Query
This will take in the user query and determine whether or not the user is wishin to search all tweet content or a specific user's content.

In [19]:
def determine_query(query):
    if(query[0] == '@' and len(query.split()) == 1):
        user_choice = raw_input(("Select your query preference",
                                "\n1. All content",
                                "\n2. User content"))
        return user_choice
    return '1'
    

In [20]:
def u_search(user_choice, query, idx, user_idx):
    if(user_choice == '1'):
		query = normalize(query)

		results = get_results_bm25(idx, query, 2.5, 0.25)
		print_results(results,25)

    if(user_choice == '2'):
		query = query[1:]
		print_results_username(user_idx, query)
    

In [21]:
# The main function that will run the search engine
def main():
    query = ""
    idx = create_inverted_index(tweet_text)
    user_idx = create_user_index(tweet_user)
    # Keep asking for a query until the user enters an "@" sign
    while query != "@":
        query = raw_input("Enter Query (@ to quit): " )
        if (query != "@"):
            stemmed_query = stem_tweet(query)
            choice = determine_query(stemmed_query)
            u_search(choice, stemmed_query, idx, user_idx)
        print ("")

# Get the name of the file to use for data
#filename = raw_input("Enter data filename: ")

# Read in the data from the csv file
docs = getTwitterDocs("twitter-data/smallData.csv")

# Create the lists that contain the tweet info
tweet_date = [ d[0] for d in docs ] #Tweet dates
tweet_user = [ d[1] for d in docs ] #Tweet users
tweet_text = [ d[2] for d in docs ] #Tweet text

#Normalize each tweet 
tweet_text = map(normalize, tweet_text)

# n - the length of the corpus
n = len(tweet_text)

# d - list with elements corresponding to the length of each document
d = [len(x.split()) for x in tweet_text]

# d_avg - the average document length of the docuemnts in the corpus
d_avg = float(sum(d) / len(d))

# Run the search engine
main()


Enter Query (@ to quit): love

Top 25 from recall set of 32 items:
	5.68 - Jen2Squared - @faithbabywear ooooh what model are you getting i have the d and love love love love it
	5.27 - ashpeckham - just got my new toy canon d love love love it
	4.01 - GrfxGuru - @mitzs hey bud np i do so love my d although i d love a d mkii more
	3.20 - PJ_King - loves twitter
	3.15 - mtgillikin - i love lebron
	3.00 - angelkim17 - @dannygokey i love you danny gokey
	3.00 - MzJill - i loved night at the museum
	2.96 - GoodPhoenix - am loving new malcolm gladwell book outliers
	2.96 - HappyWino - my kindle came and i love it
	2.87 - jackdaniels08 - @googleio yay happy place place place i love google
	2.83 - vcu451 - reading my kindle love it lee childs is good read
	2.83 - ManiKarthik - @arunbasillal i love google translator too d good day mate
	2.79 - mandanicole - how can you not love obama he makes jokes about himself
	2.68 - imusicmash - rt @jessverr i love the nerdy stanford human biology videos ma

IndexError: string index out of range